# GalSim Chromatic Objects for DEDALE
---
> Author: **Samuel Farrens [<samuel.farrens@gmail.com>]**  
> Year: **2018**

This notebook is used to generate multi-colour galaxy images with a either a Sersic, Exponential or single pixel profile using LSST and Euclid filters. SEDs are drawn from simulated spectra.

## Set-Up
---

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import galsim
from astropy import units as u
from os import environ
import pandas as pd

In the next cell you should set the path to your local GalSim installation in order to find the LSST filters.

You should also set the desired number of galaxy images and the postage stamp size for each image.

In [2]:
# Set path to GalSim
galsim_path = galsim.meta_data.share_dir
dedale_path = environ['HOME'] + '/Documents/Projects/DEDALE/spec_phot/'

# Set total number of galaxy images to generate
sample_size = 9

# Postage stamp size
postage = [41, 41]

## Survey Properties
---

The cell below sets properties for the LSST and Euclid surveys.

In [3]:
# Filter Names
lsst_filters = 'ugrizy'
euclid_filters = 'yJH'

filter_names = ['lsst_' + f for f in lsst_filters] + ['euclid_' + f for f in euclid_filters]

# Pixel Scale
lsst_pixel_size = 0.2
euclid_pixel_size = 0.3

pixel_sizes = [lsst_pixel_size for f in lsst_filters] + [euclid_pixel_size for f in euclid_filters]

# PSF FWHM
lsst_psf_fwhm = 0.7
euclid_psf_fwhm = 0.3

psf_sizes = [lsst_psf_fwhm for f in lsst_filters] + [euclid_psf_fwhm for f in euclid_filters]

## Object IDs
---

The following cell extracts the object IDs, SED types and redshifts from the file `2017-12-06_LSST_photometry.csv`.

In [4]:
object_ids, object_seds, object_z = np.genfromtxt(dedale_path + '2017-12-06_LSST_photometry.csv', dtype=str,
                                                   delimiter=',', skip_header=1, unpack=True)[:3, :sample_size]

## Survey Bandpasses
---

The following cell generates GalSim Bandpass objects using LSST and Euclid filters.

In [5]:
# Set path to LSST filters files
lsst_path = galsim_path + '/bandpasses/'
euclid_path = dedale_path + 'euclid_bandpasses/'

# Create dictionary of galsim Bandpass objects
filters = {'lsst_' + filt: galsim.Bandpass(lsst_path + 'LSST_' + filt + '.dat', 
           wave_type='nm').thin(rel_err=1e-4) for filt in lsst_filters}
filters.update({'euclid_' + filt: galsim.Bandpass(euclid_path + filt + '_NISP.Euclid.pb', 
                wave_type='nm').thin(rel_err=1e-4) for filt in euclid_filters})

del lsst_path, euclid_path

## Object SEDs
---

The following cell generates GalSim SED objects using simulated spectra.

In [6]:
def get_sed(file_name):
    
     return galsim.SED(file_name, wave_type=u.AA, flux_type=u.erg/(u.s*u.cm**2*u.AA))

# Set path to SED files
sed_path = dedale_path + 'sed_files/'

# # Create dictionary of galsim SED objects
seds = np.array([get_sed(sed_path + 'obj_{}.sed'.format(obj_id)) for obj_id in object_ids])

del sed_path

## Image Sizes
---

The following cell uses a range of half-light radius values taken from Great3 data. Galaxy images will be assigned a random flux that follows the distribution in the file `hlr_dist.npy`.

In [7]:
# Read in half light radius (HLR) distribution
hlr_dist = np.load('hlr_dist.npy')

# Get the probability density of HLR values
hlr_hist, hlr_bins = np.histogram(hlr_dist, bins=50, density=True)

# Normalise HLR hist values
hlr_hist /= hlr_hist.sum()

# Adjust bin edges to bin centres
hlr_bins = (hlr_bins[:-1] + hlr_bins[1:]) / 2.0

# Define random HLR values from PDF
random_hlr = (np.random.choice(hlr_bins, size=sample_size, p=hlr_hist) + 
              np.random.ranf(sample_size) * (hlr_bins[1] - hlr_bins[0]))

del hlr_dist, hlr_hist, hlr_bins

## Image Ellipticities
---

The following cell sets random ellipticity parameters for the images.

In [8]:
# Random rotation
object_beta = np.array([galsim.Angle(np.random.ranf() * 180.0, galsim.degrees) for obj_id in object_ids])
# Random axis ratio
object_q = np.array([np.random.ranf() for obj_id in object_ids])
# Ellipticity components
object_e1 = np.array([galsim.Shear(q=q, beta=beta).e1 for q, beta in zip(object_q, object_beta)])
object_e2 = np.array([galsim.Shear(q=q, beta=beta).e2 for q, beta in zip(object_q, object_beta)])

## Galaxy Type
---

The following cell sets random galaxy types

In [9]:
types = ('sersic', 'exponential', 'pixel')
object_types = np.array([types[np.random.randint(0, 3)] for obj_id in object_ids])

## Object Labels
---

In [10]:
labels = pd.DataFrame(data={'ID': object_ids, 'SED_Type': object_seds, 'SED': seds, 'z': object_z, 'Type' : object_types, 
                            'HLR' : random_hlr, 'PSF_Size' : psf_sizes,
                            'q' : object_q, 'beta': object_beta, 'e1' : object_e1, 'e2' : object_e2}, dtype='object')

labels

,HLR,ID,PSF_Size,SED,SED_Type,Type,beta,e1,e2,q,z
0,0.108205,256183,0.7,galsim.SED('/Users/farrens/Documents/Projects/...,223,pixel,1.22388573895 radians,-0.743823,0.618698,0.12852,0.306
1,0.410365,256184,0.7,galsim.SED('/Users/farrens/Documents/Projects/...,8788,pixel,0.312447836693 radians,0.120044,0.0865912,0.861473,0.478
2,0.116937,256326,0.7,galsim.SED('/Users/farrens/Documents/Projects/...,7377,pixel,2.09164149491 radians,-0.339007,-0.579779,0.443223,0.2196
3,0.153821,256424,0.7,galsim.SED('/Users/farrens/Documents/Projects/...,8233,pixel,2.77735993958 radians,0.553715,-0.494001,0.384803,0.3334
4,0.255894,256455,0.7,galsim.SED('/Users/farrens/Documents/Projects/...,1511,pixel,1.71535708462 radians,-0.415408,-0.123566,0.628719,1.0885
5,0.19721,256908,0.7,galsim.SED('/Users/farrens/Documents/Projects/...,4930,exponential,1.9190032196 radians,-0.368272,-0.30794,0.592708,0.6344
6,0.567888,256909,0.3,galsim.SED('/Users/farrens/Documents/Projects/...,263,exponential,1.90912162431 radians,-0.572779,-0.460019,0.391124,0.2102
7,0.457971,256957,0.3,galsim.SED('/Users/farrens/Documents/Projects/...,3922,pixel,0.871711023327 radians,-0.0415081,0.238058,0.781511,1.5873
8,0.189315,257019,0.3,galsim.SED('/Users/farrens/Documents/Projects/...,2314,pixel,0.259785518528 radians,0.180455,0.103219,0.809802,0.2626


In [11]:
labels[labels.ID == "256183"]

,HLR,ID,PSF_Size,SED,SED_Type,Type,beta,e1,e2,q,z
0,0.108205,256183,0.7,galsim.SED('/Users/farrens/Documents/Projects/...,223,pixel,1.22388573895 radians,-0.743823,0.618698,0.12852,0.306


In [12]:
labels[labels.Type == "sersic"].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

## Generate Images
---

### Extraction of Image Matrices

The following function simply draws the desired chromatic object in each of the survey filters and saves the matrices to a cube.

In [ ]:
# Define function to retrieve galaxy images
def get_images(chromatic_object):
    
    images = []

    for filter_name, pixel_size in zip(filter_names, pixel_sizes):

        # Create a galsim ImageF object with the size of the predefined postage stamp
        image = galsim.ImageF(*postage)
                
        # Draw chromatic object images in given filters
        chromatic_object.drawImage(filters[filter_name], image=image, scale=pixel_size)

        # Extract image matrix
        images.append(image._image.array)
    
    return images

### Image Generation

The following cell generates the multiband galaxy images and assignes the random half-light radius, angle and shear values defined above.

In [ ]:
def make_cube(obj):
    
#     obj_type = random_types[obj_num]
#     sed = seds[obj_num]
#     hlr = random_hlr[obj_num]
#     psf_size = psf_sizes[obj_num]
#     beta = random_beta[obj_num]
#     q = random_q[obj_num]
    
#     hlr *= 10
                
    # Generate a monochromatic object with given half light radius
    if obj.Type == 'sersic':
        mono_gal = galsim.Sersic(n=4, half_light_radius=obj.HLR).shear(q=obj.q, beta=obj.beta)
    elif obj.Type == 'exponential':
        mono_gal = galsim.Exponential(half_light_radius=obj.HLR).shear(q=obj.q, beta=obj.beta)
    elif obj.Type == 'pixel':
        mono_gal = galsim.Pixel(1., 1.)
                        
    # Create a standard chromatic object instance
    chromo_gal = mono_gal * obj.SED.atRedshift(0.0)
    
    # Convolve with PSF
#     chromo_gal = galsim.Convolve([chromo_gal, galsim.Moffat(fwhm=psf_size, beta=2.5)])
    chromo_gal = galsim.Convolve([chromo_gal, galsim.Gaussian(fwhm=obj.PSF_Size)])
            
    return get_images(chromo_gal)


In [ ]:
images = np.array([make_cube(labels.loc[i]) for i in range(sample_size)])

## Sample Outputs
---

In [ ]:
# Plot the images for a specified object
def show_images(images, obj_num):

    plt.figure(figsize=(14, 9))

    for i in range(sample_size):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[obj_num, i], interpolation='none', cmap='magma', vmin=0, 
                   vmax=np.max(images[0]))
        plt.axis('off')
        plt.title(filter_names[i], fontsize=20)

    plt.show()

### Sersic Images

In [ ]:
show_images(images, labels[labels.Type == "sersic"].index[0])

### Exponential Images

In [ ]:
show_images(images, labels[labels.Type == "exponential"].index[0])

### Pixel Images

In [ ]:
show_images(images, labels[labels.Type == "pixel"].index[0])

## Output
---

Finally this cell saves the galaxy images to a Numpy binary file called `chromatic_images.npy`.

In [ ]:
# Save imgaes to Numpy binary